<a href="https://colab.research.google.com/github/SonGoku27/Klimat/blob/main/Climat_Neon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/yandex_data/data_1.csv', sep=";")

def emissions(zxc):
    for i in ['утепление', 'температура_воздуха_в_помещении', 'скорость_воздуха', 'рост',
              'среднемесячная_температура_на_улице']:
        zxc = zxc[(zxc[i] > zxc[i].quantile(0.01)) & (zxc[i] < zxc[i].quantile(0.99))]
    return zxc

def zxc(z):
    if z >= 44:
        return 'молодой возраст'
    elif 45 <= z <= 59 :
        return 'средний возраст'
    elif z > 60:
        return 'пожилой возраст'

def qwe(x):
    if x <= 1:
        return 'мало'
    elif x == 2:
        return 'средне'
    elif x > 2:
        return 'много'

pd.set_option('display.max_columns', None)
df.dropna()
df.columns = df.columns.str.replace(' ', '_').str.lower()
df['год'] = df['год'].astype('int')
df_no_duplicates = df.drop_duplicates(
    subset=['время_года', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'способ_обогрева', 'пол',
            'ощущение_температуры_(bool)', 'предпочтительное_изменение_температуры', 'ощущение_движения_воздуха_(bool)',
            'предпочтительное_изменение_движения_воздуха', 'занавески', 'вентилятор',
            'окно', 'двери', 'отопление'],
    keep=False)
print()
df['рост'] = df['рост'].fillna(df['рост'].describe().median())
df['вес'] = df['вес'].fillna(df['вес'].describe().median())
df['оценка_комфорта'] = df['оценка_комфорта'].fillna(df['оценка_комфорта'].describe().loc['50%'])
df.loc[df['климат'] == 'Субтроп океанич', 'климат'] = 'Субтропический океанический'
df['количество_рекламаций_кат'] = [zxc(i) for i in df['количество_рекламаций']]
df['возраст_кат'] = [qwe(i) for i in df['возраст']]
df1 = emissions(df)
print(df1.describe())

print(df1)


               год     возраст  ощущение_температуры  \
count   406.000000  343.000000            406.000000   
mean   2010.926108   40.690962              0.211823   
std       0.924005   10.900835              1.112403   
min    2010.000000   22.000000             -3.000000   
25%    2010.000000   35.000000              0.000000   
50%    2011.000000   42.000000              0.000000   
75%    2012.000000   45.000000              1.000000   
max    2012.000000   65.000000              3.000000   

       ощущение_температуры_(bool)  ощущение_движения_воздуха_(bool)  \
count                   406.000000                        189.000000   
mean                      0.790640                          0.756614   
std                       0.407353                          0.430266   
min                       0.000000                          0.000000   
25%                       1.000000                          1.000000   
50%                       1.000000                          1.

In [54]:
print(df1['возраст_кат'].value_counts())
print(df1['количество_рекламаций_кат'].value_counts())

возраст_кат
много    343
Name: count, dtype: int64
Series([], Name: count, dtype: int64)


# Проверка гипотез Neon

### Есть ли взаимосвязь между количеством рекламаций и оценкой комфорта?
-  Н0: Взаимосвязь между количеством рекламаций и оценкой комфорта есть.
-  Н1: Взаимосвязи между количеством рекламаций и оценкой комфорта нет.

Есть отрицательная корреляция и она точно не случайна так как p-value = 0

In [55]:
import scipy.stats as stats

# Коэффициент корреляции Пирсона
corr, p_value = stats.pearsonr( df1['количество_рекламаций'], df1['оценка_комфорта'])
print(f'Корреляция: {round(corr, 3)}, p-value: {round(p_value, 5)}')


Корреляция: -0.492, p-value: 0.0


### Проверить гипотезу о том, что средняя оценка комфорта отличается в зависимости от страны
- Н0: Средняя оценка комфорта отличается в зависимости от страны.
- Н1: Средняя оценка комфорта не отличается в зависимости от страны.
  Исходя из этих данных можем отклонить вторую гипотезу и принять первую. Т.е,
  Средняя оценка комфорта действительно отличается в зависимости от странны. Также можем наблюдать значительно низкое среднее значение в США относительно других стран.

In [56]:
# тест на нормальность данных
stat, p_value = stats.shapiro(df1['оценка_комфорта'])
print(f"p-value: {p_value:.5f} ")


print(df1.groupby('страна')['оценка_комфорта'].mean())


groups = [group['оценка_комфорта'].values for name, group in df1.groupby('страна')]
# Применим ANOVA
f_stat, p_value = stats.f_oneway(*groups)
print(f"\n F-число: {round(f_stat,3)}, p-value: {round(p_value, 5)}")


p-value: 0.00000 
страна
Австралия    5.000000
Индия        5.050314
США          4.229310
Name: оценка_комфорта, dtype: float64

 F-число: 27.676, p-value: 0.0


Проверить гипотезу "Влияет ли способ охлаждения на оценку комфорта?"
* Н0: способ охлаждения влияет на оценку комфорта.
* Н1: способ охлаждения не влияет на оценку комфорта

Результаты отклоняют альтернативную гипотезу, следует остановиться на первой гипотезе.

In [57]:
# тест на нормальность
for name, group in df1.groupby('способ_охлаждения'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Способ охлаждения: {name}, p-value = {p:.5f}")

# Применим ANOVA
groups = [group['оценка_комфорта'].values for name, group in df.groupby('способ_охлаждения')]
f_stat, p_value = stats.f_oneway(*groups)
print(f"\nF-статистика: {f_stat:.3f}, p-value: {p_value:.5f}")

Способ охлаждения: Вентиляция, p-value = 0.00002
Способ охлаждения: Кондиционирование, p-value = 0.00000
Способ охлаждения: Смешанный, p-value = 0.00000

F-статистика: 7.944, p-value: 0.00040


Проверить гипотезу "Влияет ли пол на оценку комфорта?"
* Н0: пол не влияет на оценку комфорта.
* Н1: пол влияет на оценку комфорта

Результаты не дают нам оснований отклонять нулевую гипотезу. Следовательно пол не влияет на оценку комфорта.

In [58]:
# тест на нормальность
for name, group in df1.groupby('пол'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Пол: {name}, p-value = {p:.5f}")

# Применим метод Манна Уитни
group_male = df1[df1['пол'] == 'Мужской']['оценка_комфорта'].values
group_female = df1[df1['пол'] == 'Женский']['оценка_комфорта'].values
stat, p = stats.mannwhitneyu(group_male, group_female, alternative='two-sided')
print(f"U-статистика: {stat:.3f}, p-value: {p:.5f}")


Пол: Женский, p-value = 0.00000
Пол: Мужской, p-value = 0.00000
U-статистика: 15046.500, p-value: 0.15026


Проверить гипотезу "Влияет ли возрастная группа на оценку комфорта?"
* Н0: возрастная группа не влияет на оценку комфорта.
* Н1: возрастная группа влияет на оценку комфорта

Результаты не дают нам оснований отклонять нулевую гипотезу. Следовательно  возрастная группа не влияет на оценку комфорта.
⁉

In [63]:
df1['количество_рекламаций_кат'].value_counts()


# тест на нормальность
for name, group in df1.groupby('возраст_кат'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Возрастная группа: {name}, p-value = {p:.5f}")

# разделение на группы
young = df[df['количество_рекламаций_кат'] == 'молодая_группа']['оценка_комфорта']
middle = df[df['количество_рекламаций_кат'] == 'средняя_группа']['оценка_комфорта']
old = df[df['количество_рекламаций_кат'] == 'пожилая_группа']['оценка_комфорта']

# Kruskal–Wallis test
stat, p = stats.kruskal(young, middle, old)
print(f"Kruskal–Wallis H = {stat:.4f}, p-value = {p:.5f}")

Возрастная группа: много, p-value = 0.00000
Kruskal–Wallis H = nan, p-value = nan


<ipython-input-63-15d44e681eef>:15: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p = stats.kruskal(young, middle, old)


Проверить гипотезу "Проверить гипотезу влияет ли климат на оценку комфорта"
* Н0: климат не влияет на оценку комфорта.
* Н1: климат влияет на оценку комфорта

Результаты дают основание отклонить нулевую гипотезу и принять альтернативную.
Климат влияет на оценку комфорта,

In [ ]:
# тест на нормальность
for name, group in df1.groupby('климат'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Климат: {name}, p-value = {p:.5f}")

# Применим ANOVA
groups = [group['оценка_комфорта'].values for _, group in df1.groupby('климат')]
f_stat, p_value = stats.f_oneway(*groups)
print(f"\nF-статистика: {f_stat:.3f}, p-value: {p_value:.5f}" )


Климат: Cубтроп океанич, p-value = nan
Климат: Cубтропический океанический, p-value = 0.00000
Климат: Влажный субтропический муссонный, p-value = 0.00004
Климат: Жаркий полузасушливый, p-value = 0.00000
Климат: Субтропическое высокогорье, p-value = 0.00000
Климат: Тропическая влажная саванна, p-value = 0.00000

F-статистика: 3.094, p-value: 0.00941


<ipython-input-43-49d92fb5bcbb>:3: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  stat, p = stats.shapiro(group['оценка_комфорта'])


Проверить гипотезу "Проверить гипотезу влияет ли время года на оценку комфорта"
* Н0: время года не влияет на оценку комфорта.
* Н1: время года влияет на оценку комфорта

Результаты (p-value ≥ 0.05) дают основание принять нулевую гипотезу , и показывают что различия статистически не значимы

In [59]:
# тест на нормальность
for name, group in df1.groupby('время_года'):
    stat, p = stats.shapiro(group['оценка_комфорта'])
    print(f"Время года: {name}, p-value = {p:.5f}")
# данные не нормальные
print('данные не нормальные')
# Применим метод краскела уолиса
groups = [group['оценка_комфорта'].values for _, group in df1.groupby('время_года')]
stat, p_value = stats.kruskal(*groups)

print(f"\nKruskal-Wallis H-статистика: {stat:.3f}, p-value: {p_value:.5f}\n")



Время года: Весна, p-value = 1.00000
Время года: Зима, p-value = 0.00000
Время года: Лето, p-value = 0.00000
Время года: Осень, p-value = 1.00000
данные не нормальные

Kruskal-Wallis H-статистика: 7.607, p-value: 0.05487



/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Проверить гипотезу "Хотят ли мужчины и женщины разного изменения температуры / движения воздуха?"
* Н0: Мужчины и женщины не хотят разного изменения движения воздуха?
* Н1: Мужчины и женщины  хотят разного изменения движения воздуха?              
Результаты p ≥ 0.05  Нет статистически значимой связи между полом и предпочтениями. Придерживаемся нудевой гипотезы


In [61]:
contingency_table = pd.crosstab(df['пол'], df['предпочтительное_изменение_движения_воздуха'])
print(contingency_table)
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
print(f"X2 = {chi2:.4f}, p-value = {p:.5f}")

предпочтительное_изменение_движения_воздуха  Без изменений  Больше  Меньше
пол                                                                       
Женский                                                 94      52      12
Мужской                                                157      89       8
X2 = 4.1800, p-value = 0.12369


# Регрессионное моделирование Neon
Построить регрессионную модель температуры воздуха в помещении в зависимости от типа охлаждения и других значимых факторов. Выбор факторов - на усмотрение исполнителя.                                                                    
Выбранные факторы кроме основных:     
 * время_года
 * отопление
 * среднемесячная_температура_на_улице
 * климат
  Эти факторы были отобраны потому, что логически должны оказывать влияние на температуру внутри помещения

    
  Первая версия, с помощью линейной регрессии



In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Выбираем признаки и целевую переменную
X = df1[['способ_охлаждения', 'время_года','отопление', 'среднемесячная_температура_на_улице', 'климат' ] ]
         #'окно', 'двери'
y = df1['температура_воздуха_в_помещении']

numeric_features = ['среднемесячная_температура_на_улице']
categorical_features = [
    'время_года',
    'способ_охлаждения',
    'отопление',
    'климат'
    # 'окно', 'двери'
]



# Препроцессор: OneHotEncoder + StandardScaler
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        # ('num', StandardScaler(), numeric_features)
    ]
)

# Модельный пайплайн: сначала преобразование, потом регрессия
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regression', LinearRegression())
])

# Делим на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение
pipeline.fit(X_train, y_train)

# Предсказание
y_pred = pipeline.predict(X_test)
# Метрики
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)

R²: 0.06657855429455095
RMSE: 7.673248568397801


Вторая версия с помощью RandomForestRegressor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


# Выбираем признаки и целевую переменную
X = df1[['способ_охлаждения', 'время_года','отопление', 'среднемесячная_температура_на_улице']]
         #'окно', 'двери'
y = df1['температура_воздуха_в_помещении']

numeric_features = ['среднемесячная_температура_на_улице']
categorical_features = [
    'время_года',
    'способ_охлаждения',
    'отопление',
    #'окно', 'двери'
]

# Преобразование признаков
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), categorical_features),
    # ('num', StandardScaler(), numeric_features)
])

# Пайплайн с RandomForest
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
pipeline.fit(X_train, y_train)

# Предсказание
y_pred = pipeline.predict(X_test)

# Оценка модели
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
mae = mean_absolute_error(y_test, y_pred)

# Вывод результатов
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")


R²: 0.0692
RMSE: 7.6623
MAE: 2.6969
